In [1]:
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
import random
import requests
import pandas as pd
import re

## Laden des Datensatzes

Jetzt laden wir unseren Datensatz. Wir gehen davon aus, dass der Datensatz eine CSV-Datei mit Verkaufsdaten enthält.

Fetch Public and public Parking Data from the APIs

In [2]:

# Initialize the geolocator
geolocator = Nominatim(user_agent="parking_city_lookup")

def get_city_from_coordinates(lat, lon):
    """
    Uses reverse geocoding to determine the city based on latitude and longitude.
    """
    try:
        location = geolocator.reverse((lat, lon), exactly_one=True, language="en")
        if location:
            address_components = location.raw.get("address", {})
            return address_components.get("city") or address_components.get("town") or address_components.get("village")
    except GeocoderTimedOut:
        return None
    except Exception as e:
        print(f"Reverse geocoding failed for {lat}, {lon}: {e}")
        return None

# Function to split opening and closing times and determine open days
def parse_opening_hours(time_range):
    """
    Parses the opening hours string and returns opening time, closing time, and open days.
    """
    time_mapping = {
        "Weekdays 9AM-11PM": ("09:00", "23:00", ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday"]),
        "Weekdays 9AM-7PM": ("09:00", "19:00", ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday"]),
        "Weekdays 6AM-6PM": ("06:00", "18:00", ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday"]),
        "Weekdays 5AM-7PM": ("05:00", "19:00", ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday"]),
        "Monday-Sunday 9AM-11PM": ("09:00", "23:00", ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]),
        "Saturday 9AM-12AM": ("09:00", "24:00", ["Saturday", "Sunday"]),
        "Sunday 12AM-22AM": ("00:00", "22:00", ["Saturday", "Sunday"])
        
    }
    return time_mapping[time_range]

selected_time_range = random.choice([
    "Weekdays 9AM-11PM", "Weekdays 9AM-7PM", "Weekdays 6AM-6PM", "Weekdays 5AM-7PM", "Monday-Sunday 9AM-11PM", "Saturday 9AM-12AM", "Sunday 12AM-22AM"
])

def get_city_from_address(address):
    if address:
        parts = address.split(", ")
        
        # Check if the last part contains numbers (postal code)
        last_part = parts[-1]
        
        # If the last part contains numbers (assumed to be postal code)
        if re.search(r'\d', last_part):
            # Take the last word (city name)
            city = last_part.split()[-1]
        else:
            # Otherwise, take everything after the last comma as the city
            city = last_part
        
        return city if city else None
    return None


In [3]:


# API URL for Freiburg
api_url_freiburg = "https://api.parkendd.de/Freiburg"

# Fetch the data from the WFS API
response_freiburg = requests.get(api_url_freiburg)

if response_freiburg.status_code == 200:
    public_parking_data = response_freiburg.json()

    # Extract relevant data
    parking_data_public = []
    for site in public_parking_data['lots']:
        if site.get("purpose") and site.get("purpose") != "CAR":
            continue
        
        coords = site.get("coords", {})
        latitude, longitude = coords.get("lat"), coords.get("lng")

        opening_time, closing_time, open_days = parse_opening_hours(selected_time_range)

        parking_data_public.append({
            "public_id": site.get("id"),
            "name": site.get("name"),
            "address": site.get("address"),
            "latitude": latitude,
            "longitude": longitude,
            "price_per_hour": random.choice([3.80, 3.20, 1.60]),
            "capacity": site.get("total"),
            "available_space": site.get("free"),
            "opening_time": opening_time,
            "closing_time": closing_time,
            "open_days": open_days,  # Open days as an array
            "city": get_city_from_coordinates(latitude, longitude)  # Get city using reverse geocoding
        })
    
    # Convert to DataFrame
    df_freiburg = pd.DataFrame(parking_data_public)
    print("Public Parking Data (Freiburg):", df_freiburg.head())
else:
    print("Failed to fetch Freiburg parking data.")



Public Parking Data (Freiburg):                    public_id                name address   latitude  \
0  freiburgkollegiengebaeude    Kollegiengebäude     P19  47.993431   
1  freiburgzentrumoberwiehre  Zentrum Oberwiehre     P21  47.987991   
2         freiburgmartinstor          Martinstor     P18  47.992506   
3        freiburgschwabentor         Schwabentor     P16  47.991264   
4        freiburglandratsamt         Landratsamt     P14  48.000059   

   longitude  price_per_hour  capacity  available_space opening_time  \
0   7.847403             1.6       162               72        09:00   
1   7.871473             3.2       274              235        09:00   
2   7.847365             1.6       132              112        09:00   
3   7.856310             3.8       193              118        09:00   
4   7.857081             3.2       202              182        09:00   

  closing_time                                          open_days  \
0        23:00  [Monday, Tuesday, Wedne

In [4]:
# API URL for MobiData BW API (public parking data)
api_url_gebündelte_parkplätze_parkbauten = "https://api.mobidata-bw.de/park-api/api/public/v3/parking-sites"

# Fetch data from MobiData BW API
response_gebündelte_parkplätze_parkbauten = requests.get(api_url_gebündelte_parkplätze_parkbauten)

# Check the response and parse the JSON data
if response_gebündelte_parkplätze_parkbauten.status_code == 200:
    public_parking_data = response_gebündelte_parkplätze_parkbauten.json()
    
    # Extract relevant data (modify based on actual response structure)
    parking_data_public = []
    for site in public_parking_data['items']:
        if site.get("purpose") and site.get("purpose") != "CAR":
            continue

        latitude = float(site.get("lat", 0)) 
        longitude = float(site.get("lon", 0))  
        
        if not (47.52 <= latitude <= 49.79 and 7.50 <= longitude <= 10.49):
            continue
        
        opening_time, closing_time, open_days = parse_opening_hours(selected_time_range)

        address = site.get("address")
        city = get_city_from_address(address) if address else None

        if not city:  # If city is still missing, use geopy reverse geocoding
            city = get_city_from_coordinates(latitude, longitude)

        parking_data_public.append({
            "public_id": site.get("id"),
            "name": site.get("name"),
            "address": address,
            "latitude": latitude,
            "longitude": longitude,
            "price_per_hour": random.choice([3.80, 3.20, 1.60]),
            "capacity": site.get("capacity"),
            "available_space": site.get("realtime_free_capacity"),
            "opening_time": opening_time,
            "closing_time": closing_time,
            "open_days": open_days,  # Open days as an array
            "city": city  # Extracted or geocoded city
        })
    
    # Convert to DataFrame
    df_public_parking = pd.DataFrame(parking_data_public)
    print("Public Parking Data (MobiData BW):", df_public_parking.head())
else:
    print("Failed to fetch MobiData BW parking data.")


Reverse geocoding failed for 47.8591467, 8.1145033: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /reverse?lat=47.8591467&lon=8.1145033&format=json&accept-language=en&addressdetails=1 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x0000024CF81A0D40>, 'Connection to nominatim.openstreetmap.org timed out. (connect timeout=1)'))
Public Parking Data (MobiData BW):    public_id                         name  \
0        887  Listplatz 1 / Bahnhofstraße   
1        888          Obere Wässere 3 - 7   
2        757                Bahnhofstraße   
3        758                Am Südbahnhof   
4        759          Tannenberger Straße   

                                   address   latitude  longitude  \
0  Listplatz 1 / Bahnhofstraße, Reutlingen  48.495983   9.210331   
1          Obere Wässere 3 - 7, Reutlingen  48.488955   9.216701   
2                Bahnhofstraße, Reutlingen  48.497575   9.211077   
3          

In [5]:
df_freiburg

,public_id,name,address,latitude,longitude,price_per_hour,capacity,available_space,opening_time,closing_time,open_days,city
0,freiburgkollegiengebaeude,Kollegiengebäude,P19,47.993431,7.847403,1.6,162,72,09:00,23:00,"[Monday, Tuesday, Wednesday, Thursday, Friday,...",Freiburg im Breisgau
1,freiburgzentrumoberwiehre,Zentrum Oberwiehre,P21,47.987991,7.871473,3.2,274,235,09:00,23:00,"[Monday, Tuesday, Wednesday, Thursday, Friday,...",Freiburg im Breisgau
2,freiburgmartinstor,Martinstor,P18,47.992506,7.847365,1.6,132,112,09:00,23:00,"[Monday, Tuesday, Wednesday, Thursday, Friday,...",Freiburg im Breisgau
3,freiburgschwabentor,Schwabentor,P16,47.991264,7.856310,3.8,193,118,09:00,23:00,"[Monday, Tuesday, Wednesday, Thursday, Friday,...",Freiburg im Breisgau
4,freiburglandratsamt,Landratsamt,P14,48.000059,7.857081,3.2,202,182,09:00,23:00,"[Monday, Tuesday, Wednesday, Thursday, Friday,...",Freiburg im Breisgau
5,freiburgvolksbank,Volksbank,P3,47.998021,7.843413,3.8,186,130,09:00,23:00,"[Monday, Tuesday, Wednesday, Thursday, Friday,...",Freiburg im Breisgau
6,freiburgkonzerthaus,Konzerthaus,P2,47.995290,7.840836,1.6,421,332,09:00,23:00,"[Monday, Tuesday, Wednesday, Thursday, Friday,...",Freiburg im Breisgau
7,freiburgzurunterfuehrung,Zur Unterführung,P5,48.001286,7.844732,1.6,100,46,09:00,23:00,"[Monday, Tuesday, Wednesday, Thursday, Friday,...",Freiburg im Breisgau
8,freiburgggrafhalle,G.-Graf-Halle,P6,47.999300,7.846800,1.6,56,38,09:00,23:00,"[Monday, Tuesday, Wednesday, Thursday, Friday,...",Freiburg im Breisgau
9,freiburgschwarzwaldcity,Schwarzwald City,P9,47.997206,7.851119,1.6,391,349,09:00,23:00,"[Monday, Tuesday, Wednesday, Thursday, Friday,...",Freiburg im Breisgau


In [6]:
unique_addresses = df_freiburg["address"].unique()
print(unique_addresses)


['P19' 'P21' 'P18' 'P16' 'P14' 'P3' 'P2' 'P5' 'P6' 'P9' 'P4' 'P8' 'P13'
 'P10' 'P15' 'P22' 'P12' 'P20' 'P1']


In [7]:
df_public_parking

,public_id,name,address,latitude,longitude,price_per_hour,capacity,available_space,opening_time,closing_time,open_days,city
0,887,Listplatz 1 / Bahnhofstraße,"Listplatz 1 / Bahnhofstraße, Reutlingen",48.495983,9.210331,3.2,198.0,NaN,09:00,23:00,"[Monday, Tuesday, Wednesday, Thursday, Friday,...",Reutlingen
1,888,Obere Wässere 3 - 7,"Obere Wässere 3 - 7, Reutlingen",48.488955,9.216701,3.2,181.0,NaN,09:00,23:00,"[Monday, Tuesday, Wednesday, Thursday, Friday,...",Reutlingen
2,757,Bahnhofstraße,"Bahnhofstraße, Reutlingen",48.497575,9.211077,3.8,128.0,NaN,09:00,23:00,"[Monday, Tuesday, Wednesday, Thursday, Friday,...",Reutlingen
3,758,Am Südbahnhof,"Am Südbahnhof, Reutlingen",48.482762,9.229821,3.8,NaN,NaN,09:00,23:00,"[Monday, Tuesday, Wednesday, Thursday, Friday,...",Reutlingen
4,759,Tannenberger Straße,"Tannenberger Straße, Reutlingen",48.511669,9.204728,1.6,46.0,NaN,09:00,23:00,"[Monday, Tuesday, Wednesday, Thursday, Friday,...",Reutlingen
...,...,...,...,...,...,...,...,...,...,...,...,...
3305,18621,Parkplatz,Remchingen,48.953570,8.573547,3.2,37.0,NaN,09:00,23:00,"[Monday, Tuesday, Wednesday, Thursday, Friday,...",Remchingen
3306,18622,Parkplatz,Remchingen,48.953511,8.572800,3.2,6.0,NaN,09:00,23:00,"[Monday, Tuesday, Wednesday, Thursday, Friday,...",Remchingen
3307,18721,Parkplatz,"Sulzbach (Kr FDS), Loßburg",48.424065,8.451453,3.8,11.0,NaN,09:00,23:00,"[Monday, Tuesday, Wednesday, Thursday, Friday,...",Loßburg
3308,18882,Parkplatz,Endingen (Baden),48.144057,7.702840,3.2,8.0,NaN,09:00,23:00,"[Monday, Tuesday, Wednesday, Thursday, Friday,...",Endingen (Baden)


Data Cleaning

In [8]:
# Clean the Freiburg parking data
df_freiburg_cleaned = df_freiburg.dropna(subset=["public_id", "name", "address", "latitude", "longitude", "price_per_hour", "capacity"])  

# Clean the public parking data
df_public_parking_cleaned = df_public_parking.dropna(subset=["public_id", "name", "address", "latitude", "longitude", "price_per_hour", "capacity"]) 

# Function to fill NaN in 'available_space' with half of 'capacity'
def fill_available_space_with_half_capacity(df):
    df['available_space'] = df['available_space'].fillna(df['capacity'] / 2)
    df['available_space'] = df['available_space'].astype(int)  # Ensure integer values
    return df

# Apply the function to both datasets
df_freiburg_cleaned = fill_available_space_with_half_capacity(df_freiburg_cleaned)
df_public_parking_cleaned = fill_available_space_with_half_capacity(df_public_parking_cleaned)

# Remove duplicates based on parking ID
df_freiburg_cleaned = df_freiburg_cleaned.drop_duplicates(subset=["public_id"])
df_public_parking_cleaned = df_public_parking_cleaned.drop_duplicates(subset=["public_id"])

df_public_parking_cleaned


C:\Users\vae.tiolamon\AppData\Local\Temp\ipykernel_12276\4106914945.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['available_space'] = df['available_space'].fillna(df['capacity'] / 2)
C:\Users\vae.tiolamon\AppData\Local\Temp\ipykernel_12276\4106914945.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['available_space'] = df['available_space'].astype(int)  # Ensure integer values


,public_id,name,address,latitude,longitude,price_per_hour,capacity,available_space,opening_time,closing_time,open_days,city
0,887,Listplatz 1 / Bahnhofstraße,"Listplatz 1 / Bahnhofstraße, Reutlingen",48.495983,9.210331,3.2,198.0,99,09:00,23:00,"[Monday, Tuesday, Wednesday, Thursday, Friday,...",Reutlingen
1,888,Obere Wässere 3 - 7,"Obere Wässere 3 - 7, Reutlingen",48.488955,9.216701,3.2,181.0,90,09:00,23:00,"[Monday, Tuesday, Wednesday, Thursday, Friday,...",Reutlingen
2,757,Bahnhofstraße,"Bahnhofstraße, Reutlingen",48.497575,9.211077,3.8,128.0,64,09:00,23:00,"[Monday, Tuesday, Wednesday, Thursday, Friday,...",Reutlingen
4,759,Tannenberger Straße,"Tannenberger Straße, Reutlingen",48.511669,9.204728,1.6,46.0,23,09:00,23:00,"[Monday, Tuesday, Wednesday, Thursday, Friday,...",Reutlingen
5,483,AQUAtoll P1,"Wilfenseeweg, 74172 Neckarsulm",49.190045,9.244872,3.2,203.0,101,09:00,23:00,"[Monday, Tuesday, Wednesday, Thursday, Friday,...",Neckarsulm
...,...,...,...,...,...,...,...,...,...,...,...,...
3305,18621,Parkplatz,Remchingen,48.953570,8.573547,3.2,37.0,18,09:00,23:00,"[Monday, Tuesday, Wednesday, Thursday, Friday,...",Remchingen
3306,18622,Parkplatz,Remchingen,48.953511,8.572800,3.2,6.0,3,09:00,23:00,"[Monday, Tuesday, Wednesday, Thursday, Friday,...",Remchingen
3307,18721,Parkplatz,"Sulzbach (Kr FDS), Loßburg",48.424065,8.451453,3.8,11.0,5,09:00,23:00,"[Monday, Tuesday, Wednesday, Thursday, Friday,...",Loßburg
3308,18882,Parkplatz,Endingen (Baden),48.144057,7.702840,3.2,8.0,4,09:00,23:00,"[Monday, Tuesday, Wednesday, Thursday, Friday,...",Endingen (Baden)


Data Integration

In [9]:
# Ensure latitude and longitude columns are numeric and consistent
df_freiburg_cleaned["latitude"] = pd.to_numeric(df_freiburg_cleaned["latitude"], errors="coerce")
df_freiburg_cleaned["longitude"] = pd.to_numeric(df_freiburg_cleaned["longitude"], errors="coerce")

df_public_parking_cleaned["latitude"] = pd.to_numeric(df_public_parking_cleaned["latitude"], errors="coerce")
df_public_parking_cleaned["longitude"] = pd.to_numeric(df_public_parking_cleaned["longitude"], errors="coerce")

# Drop rows with NaN in latitude or longitude
df_freiburg_cleaned = df_freiburg_cleaned.dropna(subset=["latitude", "longitude"])
df_public_parking_cleaned = df_public_parking_cleaned.dropna(subset=["latitude", "longitude"])

# Round latitude and longitude to the same precision for matching
df_freiburg_cleaned["latitude"] = df_freiburg_cleaned["latitude"].round(6)
df_freiburg_cleaned["longitude"] = df_freiburg_cleaned["longitude"].round(6)

df_public_parking_cleaned["latitude"] = df_public_parking_cleaned["latitude"].round(6)
df_public_parking_cleaned["longitude"] = df_public_parking_cleaned["longitude"].round(6)

# Merge the DataFrames, prioritizing df_public_parking_cleaned data
df_combined = pd.merge(
    df_freiburg_cleaned,
    df_public_parking_cleaned,
    on=["latitude", "longitude"],
    how="outer",
    suffixes=("_freiburg", "_public")
)

In [10]:
# Prioritize data from df_public_parking_cleaned
for column in df_public_parking_cleaned.columns:
    if column not in ["latitude", "longitude"]:  # Exclude keys used for merging
        df_combined[column] = df_combined[column + "_public"].combine_first(df_combined[column + "_freiburg"])

# Drop redundant columns
columns_to_drop = [col for col in df_combined.columns if col.endswith("_freiburg") or col.endswith("_public")]
df_combined = df_combined.drop(columns=columns_to_drop)

df_combined

,latitude,longitude,public_id,name,address,price_per_hour,capacity,available_space,opening_time,closing_time,open_days,city
0,47.993431,7.847403,904.0,P19 Kollegiengebäude,"Platz der Universität 3, 79098 Freiburg",3.8,170.0,72.0,09:00,23:00,"[Monday, Tuesday, Wednesday, Thursday, Friday,...",Freiburg
1,47.987991,7.871473,freiburgzentrumoberwiehre,Zentrum Oberwiehre,P21,3.2,274.0,235.0,09:00,23:00,"[Monday, Tuesday, Wednesday, Thursday, Friday,...",Freiburg im Breisgau
2,47.992506,7.847365,907.0,P18 Martinstor,"Rempartstr. 13, 79098 Freiburg",3.8,132.0,112.0,09:00,23:00,"[Monday, Tuesday, Wednesday, Thursday, Friday,...",Freiburg
3,47.991264,7.856310,912.0,P16 Schwabentor,"Leo-Wohleb-Straße 4-6, 79098 Freiburg",1.6,193.0,118.0,09:00,23:00,"[Monday, Tuesday, Wednesday, Thursday, Friday,...",Freiburg
4,48.000059,7.857081,freiburglandratsamt,Landratsamt,P14,3.2,202.0,182.0,09:00,23:00,"[Monday, Tuesday, Wednesday, Thursday, Friday,...",Freiburg im Breisgau
...,...,...,...,...,...,...,...,...,...,...,...,...
2485,49.437864,8.572040,18393.0,Parkplatz,Friedrichsfeld (MA),3.8,19.0,9.0,09:00,23:00,"[Monday, Tuesday, Wednesday, Thursday, Friday,...",Friedrichsfeld (MA)
2486,48.953570,8.573547,18621.0,Parkplatz,Remchingen,3.2,37.0,18.0,09:00,23:00,"[Monday, Tuesday, Wednesday, Thursday, Friday,...",Remchingen
2487,48.953511,8.572800,18622.0,Parkplatz,Remchingen,3.2,6.0,3.0,09:00,23:00,"[Monday, Tuesday, Wednesday, Thursday, Friday,...",Remchingen
2488,48.144057,7.702840,18882.0,Parkplatz,Endingen (Baden),3.2,8.0,4.0,09:00,23:00,"[Monday, Tuesday, Wednesday, Thursday, Friday,...",Endingen (Baden)


In [11]:
# Define desired column order
desired_columns = [
    "public_id", 
    "name", 
    "address", 
    "latitude", 
    "longitude", 
    "city", 
    "price_per_hour", 
    "capacity", 
    "available_space", 
    "opening_time",
    "closing_time",
    "open_days"
]

# Reorder the columns in the combined DataFrame
df_combined = df_combined[[col for col in desired_columns if col in df_combined.columns]]

df_combined['public_id'] = df_combined['public_id'].apply(lambda x: f"PUB{random.randint(10000, 99999)}")

df_combined

,public_id,name,address,latitude,longitude,city,price_per_hour,capacity,available_space,opening_time,closing_time,open_days
0,PUB99701,P19 Kollegiengebäude,"Platz der Universität 3, 79098 Freiburg",47.993431,7.847403,Freiburg,3.8,170.0,72.0,09:00,23:00,"[Monday, Tuesday, Wednesday, Thursday, Friday,..."
1,PUB74242,Zentrum Oberwiehre,P21,47.987991,7.871473,Freiburg im Breisgau,3.2,274.0,235.0,09:00,23:00,"[Monday, Tuesday, Wednesday, Thursday, Friday,..."
2,PUB19897,P18 Martinstor,"Rempartstr. 13, 79098 Freiburg",47.992506,7.847365,Freiburg,3.8,132.0,112.0,09:00,23:00,"[Monday, Tuesday, Wednesday, Thursday, Friday,..."
3,PUB30067,P16 Schwabentor,"Leo-Wohleb-Straße 4-6, 79098 Freiburg",47.991264,7.856310,Freiburg,1.6,193.0,118.0,09:00,23:00,"[Monday, Tuesday, Wednesday, Thursday, Friday,..."
4,PUB94099,Landratsamt,P14,48.000059,7.857081,Freiburg im Breisgau,3.2,202.0,182.0,09:00,23:00,"[Monday, Tuesday, Wednesday, Thursday, Friday,..."
...,...,...,...,...,...,...,...,...,...,...,...,...
2485,PUB26113,Parkplatz,Friedrichsfeld (MA),49.437864,8.572040,Friedrichsfeld (MA),3.8,19.0,9.0,09:00,23:00,"[Monday, Tuesday, Wednesday, Thursday, Friday,..."
2486,PUB55963,Parkplatz,Remchingen,48.953570,8.573547,Remchingen,3.2,37.0,18.0,09:00,23:00,"[Monday, Tuesday, Wednesday, Thursday, Friday,..."
2487,PUB52430,Parkplatz,Remchingen,48.953511,8.572800,Remchingen,3.2,6.0,3.0,09:00,23:00,"[Monday, Tuesday, Wednesday, Thursday, Friday,..."
2488,PUB64358,Parkplatz,Endingen (Baden),48.144057,7.702840,Endingen (Baden),3.2,8.0,4.0,09:00,23:00,"[Monday, Tuesday, Wednesday, Thursday, Friday,..."


In [12]:
df_combined.to_json('public_parking.json', orient='records', indent=4)

print("DataFrame has been saved to 'public_parking.json'")

DataFrame has been saved to 'public_parking.json'
